# **Preprocessing Text**

## Apa itu Preprocessing Teks?



> **Preprocessing Teks** adalah serangkaian langkah atau teknik yang digunakan untuk membersihkan, memformat, dan mempersiapkan data teks agar sesuai untuk analisis atau pengolahan lebih lanjut oleh model pembelajaran mesin atau algoritma pemrosesan teks.


> Terdapat beberapa teknik dalam menormalisasikan teks yaitu:


1.   Punctuation process
2.   Stopword
3.   Tokenisasi
4.   Steeming






## Tahapan Preprocessing

### Masuk ke direktori data

In [ ]:
import os

In [ ]:
%cd /content/
os.mkdir('Data')

### Import recruitments yang dibutuhkan

In [ ]:
import pandas as pd
import numpy as np

### Import data yang dibutuhkan

In [ ]:
import gdown
# download data

nama_data = '/content/Data/data.csv'
gdown.download(f'https://drive.google.com/uc?id=1-IdWAIZ16LN3AqRyoKhrv3Iojggt7hDf', nama_data, quiet=False)

In [ ]:
data = pd.read_csv('/content/Data/data.csv')
data

In [ ]:
# mengambil data abstrak
data['Abstrak']

In [ ]:
# cek data kosong atau null
data.isna().sum()

In [ ]:
# hapus data NaN
data.dropna(subset=['Abstrak',"Dosen Pembimbing II"], inplace=True)

In [ ]:
# Cek kembali nilai NaN
data['Abstrak'].isna().sum()

In [ ]:
print(data['Abstrak'])

### **Punctuation process**

> **Punctuation process** merupakan proses normalisasi data yang bertujuan untuk menghilangkan tanda baca



In [ ]:
# recruitments punctuation
import string
import re

In [ ]:
# membuat kolom baru dengan nama new_abstrak untuk data baru yang dipunctuation

# data['clean_abstrak'] = data['Abstrak'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
data['clean_abstrak'] = data['Abstrak'].str.replace(r'\([^)]*\)', '', regex=True).str.lower()


In [ ]:
string.punctuation

In [ ]:
# data = data.drop('new_abstrak',axis=1)

In [ ]:
#========
# Menghilangkan angka dari kolom 'new_abstrak'
data['clean_abstrak'] = data['clean_abstrak'].str.replace('\d+', '', regex=True)


In [ ]:
data

In [ ]:
data['clean_abstrak']

### **Stopword**

> Stopwords digunakan untuk menghilangkan kata umum yang sering muncul dalam teks seperti: di, dan, atau, dari, ke, saya.



In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')

# Download kamus stop words
nltk.download('stopwords')

In [ ]:
# Inisialisasi kamus stop words dari NLTK
stop_words = set(stopwords.words('indonesian'))  # Inisialisasi kamus stop words

# Menghapus stop words dari kolom 'Abstrak'
for stop_word in stop_words:
  data['abstrak_stopword'] = data['clean_abstrak'].str.replace(rf'\b{stop_word}\b', '', regex=True) #rf untuk formating string

In [ ]:
data

In [ ]:
# data['new_abstrak']

### Tokenizing



> **Tokenizing** adalah proses memecah teks atau dokumen menjadi potongan-potongan yang lebih kecil, yang disebut token.



In [ ]:
data['tokens'] = data['abstrak_stopword'].apply(word_tokenize)

In [ ]:
# from IPython.core import error
# for i in range (len(data['tokens'])):
#   if i != error:
#     print(i,':',len(data['tokens'][i]))


In [ ]:
data

### Merge  
> Merge menaggabungkan hasil tokenizing

In [ ]:
# menggabungkan kata
data['final_abstrak'] = data['tokens'].apply(lambda x: ' '.join(x))

In [ ]:
data

In [ ]:
data['final_abstrak']

## VSM

### Binary

> Binary merupakan sebuah pengolahan data yang bertujuan agar dapat digunakan oleh algoritma machine learning. Binary akan mengkonversi kata unik menjadi vektor binner yaitu 1 0.



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#coun_vect = CountVectorizer()
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(data['final_abstrak'])
count_array = count_matrix.toarray()
df_countMatrix = pd.DataFrame(data=count_array,columns = vectorizer.vocabulary_.keys())

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Inisialisasi OneHotEncoder
encoder = OneHotEncoder(drop="if_binary")

# Melakukan one-hot encoding pada kolom 'final_abstrak'
one_hot_encoded = encoder.fit_transform(df_countMatrix)

# Mendapatkan nama fitur (kolom)
fitur_names = encoder.get_feature_names_out(input_features=df_countMatrix.columns)

# Membuat DataFrame dari hasil one-hot encoding
one_hot_df = pd.DataFrame(one_hot_encoded.toarray(), columns=fitur_names)


In [ ]:
one_hot_df

In [ ]:
one_hot_df.to_csv('binary.csv', index=False)

### Term Freq

> Term Freq adalah konsep yang digunakan dalam pemrosesan teks dan analisis teks untuk mengukur sejauh mana suatu kata atau term muncul dalam sebuah dokumen atau koleksi dokumen. Term frequency menggambarkan seberapa sering sebuah kata muncul dalam teks relatif terhadap total kata dalam dokumen tersebut.  

> \begin{equation}
tf(t,d) = n(t,d)
\end{equation}  
- tf(t,d) adalah term frekuensi dari kata t dalam dokumen d
- n(t,d) adalah jumlah kemunculan kata t dalam dokumen d







In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Inisialisasi DataFrame untuk Term Frequency (TF)
df = pd.DataFrame(data)

# Inisialisasi CountVectorizer
vectorizer = CountVectorizer()

# Melakukan transformasi TF pada kolom 'final_abstrak'
tf_matrix = vectorizer.fit_transform(df['final_abstrak'])

# Membuat DataFrame dari hasil TF
tf_df = pd.DataFrame(tf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Cetak DataFrame TF
tf_df


In [ ]:
tf_df.to_csv('tf_df.csv', index=False)

### Log Freq

>  Log Freq adalah perhitungan bobot kata atau istilah (term weights) dalam representasi vektor dokumen. Ini bertujuan untuk mengurangi dampak kata-kata yang muncul sangat sering dalam koleksi dokumen, sehingga kata-kata tersebut tidak mendominasi bobot vektor.  

> \begin{equation}
\log_{10}tf(t,d) = \log_{10}\left(\frac{n(t,d)}{|d|}\right)
\end{equation}  
  
- log 10 tf(t,d) is the log term frequency of word t in document d
- n(t,d) is the number of occurrences of word t in document d  
- ∣d∣ is the number of words in document d



In [ ]:
import numpy as np

In [ ]:
# Inisialisasi CountVectorizer
vectorizer = CountVectorizer()

# Melakukan transformasi TF pada kolom 'final_abstrak'
tf_matrix = vectorizer.fit_transform(df['final_abstrak'])

# Menghitung log-TF dengan logaritma natural (ln)
log_tf_matrix = np.log1p(tf_matrix)

# Membuat DataFrame dari hasil log-TF
log_tf_df = pd.DataFrame(log_tf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

# Cetak DataFrame log-TF
log_tf_df

In [ ]:
# Menyimpan DataFrame ke dalam berkas CSV
log_tf_df.to_csv('log_tf.csv', index=False)

### TF-IDF



> metode yang digunakan dalam pemrosesan bahasa alami (Natural Language Processing - NLP) dan pengambilan informasi (Information Retrieval) untuk mengukur seberapa penting suatu kata dalam suatu dokumen atau kumpulan dokumen. Tujuannya adalah untuk memberikan bobot yang lebih tinggi kepada kata-kata yang penting dalam representasi teks.  

> \begin{equation}
tfidf(t,d) = tf(t,d) \cdot idf(t)
\end{equation}  

- tfidf(t,d) adalah tfidf dari kata t dalam dokumen d
- tf(t,d) adalah term frekuensi dari kata t dalam dokumen d
- idf(t) adalah inverse document frequency dari kata t



In [ ]:
%pwd

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Inisialisasi TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Melakukan transformasi TF-IDF pada kolom 'final_abstrak'
tfidf_matrix = tfidf_vectorizer.fit_transform(df['final_abstrak'])

# Membuat DataFrame dari hasil TF-IDF
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Cetak DataFrame TF-IDF
tfidf_df

In [ ]:
# Menyimpan DataFrame ke dalam berkas CSV
tfidf_df.to_csv('tfidf.csv', index=False)

In [ ]:
data.columns

In [ ]:
data = data[['Judul', 'Penulis', 'Dosen Pembimbing I', 'Dosen Pembimbing II',
            'Abstrak', 'clean_abstrak', 'abstrak_stopword', 'tokens', 'final_abstrak', 'Label']]
data